# #0 Table of Content
1. Import packages and data
2. Data cleanse
3. r
4. r
5. f

# #1 Import packages, data and creating functions

In [26]:
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

In [61]:
def sentiment_scores(sentence):
# Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
# polarity_scores method of SentimentIntensityAnalyzer
# oject gives a sentiment dictionary.
# which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    print("Sentence Overall Rated As", end = " ")
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.5 :
        print("Positive")
    elif sentiment_dict['compound'] <= - 0 :
        print("Negative")
    else :
        print("Neutral")

def pre_process(text):

    # remove special characters and digits
    text=re.sub("(\\d|\\W|_)+"," ", str(text))

    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # change all to lowercase
    text=text.lower()
    
    return str(text)

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

# #2 Data Cleansing

In [23]:
review_table   = pd.read_csv("../asset/etlReview.csv")

In [24]:
review_table

,review_id,username,product_id,date,rating,content,location
0,6ef347c0-c603-422a-b8d1-c4b96bed0207,i*****b,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-09-02 01:13:00,5,Best buy ever\r\nit looks great works great\r\...,NaN
1,af3ccbed-1865-4492-88e3-723e9dda0de9,jessylim70,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-07-21 23:33:00,5,Item received in good condition.\r\nBought dur...,NaN
2,d17ebe72-919d-4c2a-a230-88119aac725c,s*****b,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-05-08 19:37:00,5,Value For Money: yes\r\nBest Feature(s): comfo...,NaN
3,9b72986c-7525-425e-89c6-e27d7e18af50,a*****w,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-06-13 11:03:00,5,Impressed with the quality and looks really ni...,NaN
4,3af89ba2-29c8-4adb-bb3a-cf9dd3ef2255,j*****n,912d838b-5e84-4aef-a0dd-bb23f44e5913,2023-10-06 12:56:00,5,Value For Money: yes\r\nBest Feature(s): the w...,NaN
...,...,...,...,...,...,...,...
155248,6d9aedbd-acf4-43c3-aefc-6a7eae22387b,nnumbs,e634b9ba-08e3-4d00-997b-ee4a3437b448,2021-08-05 11:38:00,5,NaN,NaN
155249,5a7c4ed5-a77d-4c77-92da-633f20147265,yld7zt3fab,e634b9ba-08e3-4d00-997b-ee4a3437b448,2021-11-14 08:31:00,5,NaN,NaN
155250,3864b6fe-2dde-4ebf-a7d9-4ab2ab54e53e,elischan93,8d69cbbc-7c81-41fd-8414-fc22895f3f6b,2023-05-03 20:43:00,5,Value For Money: yes\r\nBest Feature(s): Nice ...,NaN
155251,dca0ff22-40cf-48fc-a222-cf30d5e00388,yxfxrblqz5,8d69cbbc-7c81-41fd-8414-fc22895f3f6b,2023-03-25 18:24:00,5,Value For Money: song system is good .buy again,NaN


In [54]:
reviews_all = review_table['content'].apply(lambda x:pre_process(x))

english_stop_words = stopwords.words('english')

no_stop_words = remove_stop_words(reviews_all)
no_stop_words

['best buy ever looks great works great sound quality amazing connects via bluetooth easily free charging wire given foldable adjustable cushion really soft top headband part rubber keeps sliding',
 'item received good condition bought flash sales try yet look good price light weight bulky fold come bag',
 'value money yes best feature comfortable wear performance good received fast connect via wireless comfortable wear extra wire given equipment without bluetooth',
 'impressed quality looks really nice shade white wanted',
 'value money yes best feature weight performance good excited try performance ordered days ago hereee wait try love colour lightweight',
 'value money yes performance feels great best feature sounds good first buy value money hope last sounds good arrive fast well',
 'best feature better using music asmr random shows k drama decent still great cz scroll youtube forgot headphones really comfortable sounds feels earpiece noise cancelation best istg looking noise canc

# TESTING OF CODES

In [14]:
sentiment = SentimentIntensityAnalyzer()
text_1 = review_table['content'][0]
text_2 =  review_table['content'][1]
sent_1 = sentiment.polarity_scores(text_1)
sent_2 = sentiment.polarity_scores(text_2)
print("Sentiment of text 1:", sent_1)
print("Sentiment of text 2:", sent_2)

Sentiment of text 1: {'neg': 0.0, 'neu': 0.607, 'pos': 0.393, 'compound': 0.9749}
Sentiment of text 2: {'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'compound': 0.7003}


In [15]:
review_table['content'][0]

'Best buy ever\r\nit looks great works great\r\nthe sound quality is amazing, and it connects via bluetooth easily\r\na free charging wire was given \r\nits foldable and adjustable\r\nthe cushion is really soft, the top headband part has rubber, keeps it from sliding'

In [64]:
# Sort
sorted_counts = review_table.sort_values(by='rating', ascending=True).reset_index()
top = sorted_counts.head(20)
top['content'][2]

'The shift sometimes doesnt work'

In [69]:
reviews_all = review_table['content'].apply(lambda x:pre_process(x))

#text = "123..3213.2321 hehy you YOU hah /\n itâ€™s okayyy... ðŸ‘ðŸ»  is okay ðŸ‘ŒðŸ¼"
text = 'Performance: decent\r\nValue For Money: yes\r\nBest Feature(s): ?\r\nSpoilt in 2 weeks. Usb stopped connecting entirely for no reason'

sentiment_scores('looks good price bought')
#reviews_all

Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}
sentence was rated as  0.0 % Negative
sentence was rated as  50.8 % Neutral
sentence was rated as  49.2 % Positive
Sentence Overall Rated As Neutral


# #References

In [9]:
# Download the English stopwords file
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rache\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
